# 🦷 YOLOv8 Dental Detection - Simple & Stable

Train YOLOv8 để khoanh vùng bệnh lý răng miệng

**Dataset**: Oral Diseases (7 classes)

**Output**: dental_detection_yolo.pt

## Bước 1: Install & Import

In [ ]:
import subprocess
import sys

# Install dependencies với đúng version
print("🔧 Installing dependencies...\n")

subprocess.run([sys.executable, "-m", "pip", "uninstall", "-y", "opencv-python", "opencv-python-headless"], 
               capture_output=True, check=False)

print("📥 Installing opencv-python==4.8.0.76...")
subprocess.run([sys.executable, "-m", "pip", "install", "opencv-python==4.8.0.76", "-q"], check=True)

print("📥 Installing ultralytics...")
subprocess.run([sys.executable, "-m", "pip", "install", "ultralytics", "-q"], check=True)

print("\n✅ Installation complete!\n")

# Import all dependencies
from ultralytics import YOLO
import os
import cv2
import torch
import shutil
from pathlib import Path

# Disable warnings
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

# Fix PyTorch 2.6+ weights_only issue
torch.serialization.add_safe_globals([object])

print(f"✅ OpenCV: {cv2.__version__}")
print(f"✅ PyTorch: {torch.__version__}")
print(f"✅ Ready to train!")

# Check OpenCV version
if cv2.__version__.startswith('4.8'):
    print("\n🎉 OpenCV version OK!")
else:
    print(f"\n⚠️  WARNING: OpenCV {cv2.__version__} detected")
    print("    Expected 4.8.x for best compatibility")

## Bước 2: Check Dataset

In [ ]:
# Dataset path
DATASET_PATH = '/kaggle/input/oral-diseases'

# Check dataset
if os.path.exists(DATASET_PATH):
    classes = [d for d in os.listdir(DATASET_PATH) if os.path.isdir(os.path.join(DATASET_PATH, d))]
    print(f"✅ Dataset found: {len(classes)} classes\n")
    for cls in classes:
        print(f"  📁 {cls}")
else:
    print("❌ Dataset not found!")
    print("👉 Add 'oral-diseases' dataset to notebook")

## Bước 3: Convert to YOLO Format

In [ ]:
# Simple class mapping
class_mapping = {
    'Data caries': 0,
    'Mouth Ulcer': 1,
    'Tooth Discoloration': 2,
    'hypodontia': 3,
    'Gingivitis': 4,
    'Calculus': 5,
    'Caries_Gingivitus_ToothDiscoloration_Ulcer-yolo_annotated-Dataset': 6
}

# Output path
YOLO_DATASET = '/kaggle/working/dental_yolo'
os.makedirs(f'{YOLO_DATASET}/images/train', exist_ok=True)
os.makedirs(f'{YOLO_DATASET}/images/val', exist_ok=True)
os.makedirs(f'{YOLO_DATASET}/labels/train', exist_ok=True)
os.makedirs(f'{YOLO_DATASET}/labels/val', exist_ok=True)

train_count = 0
val_count = 0
MAX_PER_CLASS = 1200  # Giảm xuống cho train nhanh hơn

print("🔄 Converting dataset...\n")

for class_name, class_id in class_mapping.items():
    class_path = os.path.join(DATASET_PATH, class_name)
    
    if not os.path.exists(class_path):
        continue
    
    # Collect images
    images = []
    for root, dirs, files in os.walk(class_path):
        for file in files:
            if file.lower().endswith(('.jpg', '.jpeg', '.png')):
                img_path = os.path.join(root, file)
                # Quick validation
                try:
                    img = cv2.imread(img_path)
                    if img is not None and img.size > 0:
                        images.append(img_path)
                except:
                    pass
    
    # Limit
    images = images[:MAX_PER_CLASS]
    print(f"📷 {class_name}: {len(images)} images")
    
    # 80-20 split
    split = int(len(images) * 0.8)
    train_imgs = images[:split]
    val_imgs = images[split:]
    
    # Process train
    for img_path in train_imgs:
        try:
            dst_img = f'{YOLO_DATASET}/images/train/{class_name}_{train_count}.jpg'
            shutil.copy2(img_path, dst_img)
            
            # Create label (whole image as bounding box)
            label_file = f'{YOLO_DATASET}/labels/train/{class_name}_{train_count}.txt'
            with open(label_file, 'w') as f:
                f.write(f"{class_id} 0.5 0.5 1.0 1.0\n")
            
            train_count += 1
        except:
            pass
    
    # Process val
    for img_path in val_imgs:
        try:
            dst_img = f'{YOLO_DATASET}/images/val/{class_name}_{val_count}.jpg'
            shutil.copy2(img_path, dst_img)
            
            label_file = f'{YOLO_DATASET}/labels/val/{class_name}_{val_count}.txt'
            with open(label_file, 'w') as f:
                f.write(f"{class_id} 0.5 0.5 1.0 1.0\n")
            
            val_count += 1
        except:
            pass

print(f"\n✅ Conversion done!")
print(f"  📊 Train: {train_count} images")
print(f"  📊 Val: {val_count} images")

## Bước 4: Create YAML Config

In [ ]:
yaml_content = f"""path: {YOLO_DATASET}
train: images/train
val: images/val

nc: 7
names:
  0: Data caries
  1: Mouth Ulcer
  2: Tooth Discoloration
  3: hypodontia
  4: Gingivitis
  5: Calculus
  6: Caries_Gingivitus_ToothDiscoloration_Ulcer
"""

yaml_path = '/kaggle/working/data.yaml'
with open(yaml_path, 'w') as f:
    f.write(yaml_content)

print("✅ YAML config created")

## Bước 5: Train YOLOv8

In [ ]:
print("🚀 Starting training...\n")

# Load model
model = YOLO('yolov8n.pt')

# Train - SIMPLE CONFIG
results = model.train(
    data=yaml_path,
    epochs=25,
    imgsz=640,
    batch=16,
    device=0,
    name='dental',
    patience=8,
    workers=0,  # No multiprocessing to avoid OpenCV errors
    amp=False,  # Disable AMP for OpenCV compatibility
    verbose=True
)

print("\n✅ Training completed!")

## Bước 6: Test Model

In [ ]:
import matplotlib.pyplot as plt

# Load best model
best_model = YOLO('/kaggle/working/runs/detect/dental/weights/best.pt')

# Test on validation images
val_images = list(Path(f'{YOLO_DATASET}/images/val').glob('*.jpg'))[:6]

fig, axes = plt.subplots(2, 3, figsize=(15, 10))
axes = axes.flatten()

for idx, img_path in enumerate(val_images):
    results = best_model(str(img_path), verbose=False)
    annotated = results[0].plot()
    
    axes[idx].imshow(cv2.cvtColor(annotated, cv2.COLOR_BGR2RGB))
    axes[idx].set_title(f'{len(results[0].boxes)} detections')
    axes[idx].axis('off')

plt.tight_layout()
plt.show()

print("✅ Test completed!")

## Bước 7: Export Model

In [ ]:
# Copy model
src = '/kaggle/working/runs/detect/dental/weights/best.pt'
dst = '/kaggle/working/dental_detection_yolo.pt'

if os.path.exists(src):
    shutil.copy2(src, dst)
    size_mb = os.path.getsize(dst) / (1024 * 1024)
    print(f"✅ Model exported: dental_detection_yolo.pt")
    print(f"📦 Size: {size_mb:.2f} MB")
else:
    print("❌ Model not found!")

# Save classes
with open('/kaggle/working/yolo_classes.txt', 'w') as f:
    for name, id in class_mapping.items():
        f.write(f"{id}: {name}\n")

print("\n" + "="*60)
print("🎉 DONE!")
print("="*60)
print("\n📥 Download từ Output tab:")
print("  1. dental_detection_yolo.pt (model chính)")
print("  2. yolo_classes.txt (danh sách classes)")
print("\n💾 Copy dental_detection_yolo.pt vào folder models/")

# 🦷 YOLOv8 Dental Detection - Simple & Stable

Train YOLOv8 detection model đơn giản như train CNN

**Dataset**: Oral Diseases (7 classes)

**Output**: dental_detection_yolo.pt

## Bước 1: Install Dependencies

In [ ]:
# Fix OpenCV version conflict
!pip uninstall opencv-python opencv-python-headless -y -q
!pip install opencv-python==4.8.1.78 -q
!pip install ultralytics -q

print("✅ Dependencies installed!")

## Bước 2: Import & Setup

In [ ]:
from ultralytics import YOLO
import os

# Disable warnings
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

# Fix PyTorch 2.6+ weights_only issue
import torch
torch.serialization.add_safe_globals([object])  # Allow all

print("✅ Imports ready")
print(f"PyTorch: {torch.__version__}")

## Bước 3: Prepare Dataset

In [ ]:
# Dataset path
DATASET_PATH = '/kaggle/input/oral-diseases'

# Check dataset
if os.path.exists(DATASET_PATH):
    classes = [d for d in os.listdir(DATASET_PATH) if os.path.isdir(os.path.join(DATASET_PATH, d))]
    print(f"✅ Dataset found: {len(classes)} classes")
    for cls in classes:
        print(f"  - {cls}")
else:
    print("❌ Dataset not found!")
    print("👉 Add 'oral-diseases' dataset to notebook")

## Bước 4: Create YOLO Dataset

In [ ]:
import cv2
import shutil
from pathlib import Path

# Simple class mapping
class_mapping = {
    'Data caries': 0,
    'Mouth Ulcer': 1,
    'Tooth Discoloration': 2,
    'hypodontia': 3,
    'Gingivitis': 4,
    'Calculus': 5,
    'Caries_Gingivitus_ToothDiscoloration_Ulcer-yolo_annotated-Dataset': 6
}

# Output path
YOLO_DATASET = '/kaggle/working/dental_yolo'
os.makedirs(f'{YOLO_DATASET}/images/train', exist_ok=True)
os.makedirs(f'{YOLO_DATASET}/images/val', exist_ok=True)
os.makedirs(f'{YOLO_DATASET}/labels/train', exist_ok=True)
os.makedirs(f'{YOLO_DATASET}/labels/val', exist_ok=True)

train_count = 0
val_count = 0
MAX_PER_CLASS = 1500  # Limit for faster training

print("🔄 Converting dataset...\n")

for class_name, class_id in class_mapping.items():
    class_path = os.path.join(DATASET_PATH, class_name)
    
    if not os.path.exists(class_path):
        continue
    
    # Collect images
    images = []
    for root, dirs, files in os.walk(class_path):
        for file in files:
            if file.lower().endswith(('.jpg', '.jpeg', '.png')):
                img_path = os.path.join(root, file)
                images.append(img_path)
    
    # Limit
    images = images[:MAX_PER_CLASS]
    print(f"📷 {class_name}: {len(images)} images")
    
    # 80-20 split
    split = int(len(images) * 0.8)
    train_imgs = images[:split]
    val_imgs = images[split:]
    
    # Process train
    for img_path in train_imgs:
        try:
            dst_img = f'{YOLO_DATASET}/images/train/{class_name}_{train_count}.jpg'
            shutil.copy2(img_path, dst_img)
            
            # Create label (whole image)
            label_file = f'{YOLO_DATASET}/labels/train/{class_name}_{train_count}.txt'
            with open(label_file, 'w') as f:
                f.write(f"{class_id} 0.5 0.5 1.0 1.0\n")
            
            train_count += 1
        except:
            pass
    
    # Process val
    for img_path in val_imgs:
        try:
            dst_img = f'{YOLO_DATASET}/images/val/{class_name}_{val_count}.jpg'
            shutil.copy2(img_path, dst_img)
            
            label_file = f'{YOLO_DATASET}/labels/val/{class_name}_{val_count}.txt'
            with open(label_file, 'w') as f:
                f.write(f"{class_id} 0.5 0.5 1.0 1.0\n")
            
            val_count += 1
        except:
            pass

print(f"\n✅ Conversion done!")
print(f"  Train: {train_count} images")
print(f"  Val: {val_count} images")

## Bước 5: Create YAML Config

In [ ]:
yaml_content = f"""path: {YOLO_DATASET}
train: images/train
val: images/val

nc: 7
names:
  0: Data caries
  1: Mouth Ulcer
  2: Tooth Discoloration
  3: hypodontia
  4: Gingivitis
  5: Calculus
  6: Caries_Gingivitus_ToothDiscoloration_Ulcer
"""

yaml_path = '/kaggle/working/data.yaml'
with open(yaml_path, 'w') as f:
    f.write(yaml_content)

print("✅ YAML config created")

## Bước 6: Train YOLOv8

In [ ]:
print("🚀 Starting training...\n")

# Load model
model = YOLO('yolov8n.pt')

# Train - SIMPLE CONFIG
results = model.train(
    data=yaml_path,
    epochs=20,
    imgsz=640,
    batch=16,
    device=0,
    name='dental',
    patience=5,
    amp=False,  # Disable AMP to avoid OpenCV errors
    verbose=True
)

print("\n✅ Training completed!")

## Bước 7: Test Model

In [ ]:
import matplotlib.pyplot as plt

# Load best model
best_model = YOLO('/kaggle/working/runs/detect/dental/weights/best.pt')

# Test on validation images
val_images = list(Path(f'{YOLO_DATASET}/images/val').glob('*.jpg'))[:6]

fig, axes = plt.subplots(2, 3, figsize=(15, 10))
axes = axes.flatten()

for idx, img_path in enumerate(val_images):
    results = best_model(str(img_path), verbose=False)
    annotated = results[0].plot()
    
    axes[idx].imshow(cv2.cvtColor(annotated, cv2.COLOR_BGR2RGB))
    axes[idx].set_title(f'{len(results[0].boxes)} detections')
    axes[idx].axis('off')

plt.tight_layout()
plt.show()

print("✅ Test completed!")

## Bước 8: Export Model

In [ ]:
# Copy model
src = '/kaggle/working/runs/detect/dental/weights/best.pt'
dst = '/kaggle/working/dental_detection_yolo.pt'

if os.path.exists(src):
    shutil.copy2(src, dst)
    size_mb = os.path.getsize(dst) / (1024 * 1024)
    print(f"✅ Model exported: dental_detection_yolo.pt")
    print(f"📦 Size: {size_mb:.2f} MB")
else:
    print("❌ Model not found!")

# Save classes
with open('/kaggle/working/yolo_classes.txt', 'w') as f:
    for name, id in class_mapping.items():
        f.write(f"{id}: {name}\n")

print("\n🎉 DONE!")
print("📥 Download dental_detection_yolo.pt from Output tab")

# 🦷 YOLOv8 Detection Training - Khoanh vùng bệnh lý răng miệng

Train YOLOv8 model để **phát hiện & khoanh vùng** bệnh lý trên ảnh răng

**Dataset**: Oral Diseases từ Kaggle (7 classes)

**GPU**: 2x T4 (Kaggle) → ~15-20 phút hoàn thành

**Output**: dental_detection_yolo.pt

## Bước 1: Cài đặt Dependencies

In [ ]:
import sys
import subprocess
import os

print("🔧 Force installing dependencies...\n")

# Force uninstall
subprocess.call([sys.executable, "-m", "pip", "uninstall", "-y", "opencv-python", "opencv-python-headless", "-q"])

# Force install correct versions
print("📥 Installing opencv-python==4.6.0.66...")
subprocess.check_call([sys.executable, "-m", "pip", "install", "--force-reinstall", "--no-deps", "opencv-python==4.6.0.66", "-q"])

print("📥 Installing ultralytics==8.0.120...")
subprocess.check_call([sys.executable, "-m", "pip", "install", "--force-reinstall", "--no-deps", "ultralytics==8.0.120", "-q"])

print("\n✅ Dependencies installed!")

## Bước 2: Verify Dependencies

In [ ]:
import cv2
import torch
from ultralytics import YOLO
from ultralytics.nn.tasks import DetectionModel
import tensorflow as tf
from pathlib import Path
import matplotlib.pyplot as plt
import shutil

print(f"OpenCV: {cv2.__version__}")
print(f"PyTorch: {torch.__version__}")
print(f"TensorFlow: {tf.__version__}")
print(f"YOLO: Ready")

if cv2.__version__.startswith('4.6'):
    print("\n✅ All dependencies OK!")
else:
    print(f"\n❌ Wrong OpenCV version: {cv2.__version__}")
    print("⚠️  RESTART KERNEL and run cell 1 again!")

## Bước 3: Load Dataset

In [ ]:
DATASET_PATH = '/kaggle/input/oral-diseases'

print("📁 Checking dataset...")
if os.path.exists(DATASET_PATH):
    classes = [d for d in os.listdir(DATASET_PATH) if os.path.isdir(os.path.join(DATASET_PATH, d))]
    print(f"✅ Dataset found: {len(classes)} classes")
    for cls in classes:
        print(f"  - {cls}")
else:
    print(f"❌ Dataset not found at {DATASET_PATH}")

## Bước 4: Convert to YOLO Format

In [ ]:
# Class mapping - 7 classes
class_mapping = {
    'Data caries': 0,
    'Mouth Ulcer': 1,
    'Tooth Discoloration': 2,
    'hypodontia': 3,
    'Gingivitis': 4,
    'Calculus': 5,
    'Caries_Gingivitus_ToothDiscoloration_Ulcer-yolo_annotated-Dataset': 6
}

print(f"📊 Class mapping (7 classes):\n")
for name, idx in class_mapping.items():
    print(f"  {idx}: {name}")

# Helper function
def is_valid_image(img_path):
    try:
        img = cv2.imread(str(img_path))
        return img is not None and img.size > 0
    except:
        return False

# Create YOLO structure
YOLO_DATASET = '/kaggle/working/dental_yolo_dataset'
os.makedirs(f'{YOLO_DATASET}/images/train', exist_ok=True)
os.makedirs(f'{YOLO_DATASET}/images/val', exist_ok=True)
os.makedirs(f'{YOLO_DATASET}/labels/train', exist_ok=True)
os.makedirs(f'{YOLO_DATASET}/labels/val', exist_ok=True)

train_count = 0
val_count = 0
class_counts = {name: 0 for name in class_mapping.keys()}
skipped = 0

# LIMIT: Chỉ lấy tối đa 2000 ảnh trên class để train nhanh
MAX_IMAGES_PER_CLASS = 2000

print("\n🔄 Processing classes...\n")

for class_name, class_id in class_mapping.items():
    class_path = os.path.join(DATASET_PATH, class_name)
    
    if class_name == 'Caries_Gingivitus_ToothDiscoloration_Ulcer-yolo_annotated-Dataset':
        # Pre-annotated YOLO format
        print(f"📍 Processing pre-annotated YOLO class: {class_name}")
        
        yolo_data_path = os.path.join(class_path, class_path, 'Data')
        
        if os.path.exists(yolo_data_path):
            # Train images
            train_imgs = list(Path(f'{yolo_data_path}/images/train').glob('*.jpg'))
            train_imgs += list(Path(f'{yolo_data_path}/images/train').glob('*.png'))
            train_imgs = train_imgs[:MAX_IMAGES_PER_CLASS]  # LIMIT
            
            for src_img in train_imgs:
                if not is_valid_image(src_img):
                    skipped += 1
                    continue
                
                dst_img = os.path.join(YOLO_DATASET, 'images/train', f'yolo_{train_count}.jpg')
                shutil.copy2(src_img, dst_img)
                
                label_name = src_img.stem + '.txt'
                src_label = os.path.join(f'{yolo_data_path}/labels/train', label_name)
                dst_label = os.path.join(YOLO_DATASET, 'labels/train', f'yolo_{train_count}.txt')
                
                if os.path.exists(src_label):
                    with open(src_label, 'r') as f:
                        lines = f.readlines()
                    
                    with open(dst_label, 'w') as f:
                        for line in lines:
                            parts = line.strip().split()
                            if len(parts) >= 5:
                                parts[0] = str(class_id)
                                f.write(' '.join(parts) + '\n')
                    
                    train_count += 1
            
            # Val images
            val_imgs = list(Path(f'{yolo_data_path}/images/val').glob('*.jpg'))
            val_imgs += list(Path(f'{yolo_data_path}/images/val').glob('*.png'))
            val_imgs = val_imgs[:int(MAX_IMAGES_PER_CLASS * 0.2)]  # LIMIT
            
            for src_img in val_imgs:
                if not is_valid_image(src_img):
                    skipped += 1
                    continue
                
                dst_img = os.path.join(YOLO_DATASET, 'images/val', f'yolo_{val_count}.jpg')
                shutil.copy2(src_img, dst_img)
                
                label_name = src_img.stem + '.txt'
                src_label = os.path.join(f'{yolo_data_path}/labels/val', label_name)
                dst_label = os.path.join(YOLO_DATASET, 'labels/val', f'yolo_{val_count}.txt')
                
                if os.path.exists(src_label):
                    with open(src_label, 'r') as f:
                        lines = f.readlines()
                    
                    with open(dst_label, 'w') as f:
                        for line in lines:
                            parts = line.strip().split()
                            if len(parts) >= 5:
                                parts[0] = str(class_id)
                                f.write(' '.join(parts) + '\n')
                    
                    val_count += 1
            
            print(f"  ✅ {class_name}: Processed")
    
    elif os.path.exists(class_path):
        # Classification format
        images = []
        
        for root, dirs, files in os.walk(class_path):
            for file in files:
                if file.lower().endswith(('.jpg', '.jpeg', '.png')):
                    img_path = os.path.join(root, file)
                    if is_valid_image(img_path):
                        images.append(img_path)
                    else:
                        skipped += 1
        
        # LIMIT: Chỉ lấy tối đa MAX_IMAGES_PER_CLASS ảnh
        images = images[:MAX_IMAGES_PER_CLASS]
        
        class_counts[class_name] = len(images)
        print(f"📷 Converting '{class_name}': {len(images)} images")
        
        # 80-20 split
        split_idx = int(len(images) * 0.8)
        train_images = images[:split_idx]
        val_images = images[split_idx:]
        
        # Copy train
        for i, img_path in enumerate(train_images):
            dst_img = os.path.join(YOLO_DATASET, 'images/train', f'{class_name}_{i}.jpg')
            shutil.copy2(img_path, dst_img)
            
            label_file = os.path.join(YOLO_DATASET, 'labels/train', f'{class_name}_{i}.txt')
            with open(label_file, 'w') as f:
                f.write(f"{class_id} 0.5 0.5 1.0 1.0\n")
            train_count += 1
        
        # Copy val
        for i, img_path in enumerate(val_images):
            dst_img = os.path.join(YOLO_DATASET, 'images/val', f'{class_name}_{i}.jpg')
            shutil.copy2(img_path, dst_img)
            
            label_file = os.path.join(YOLO_DATASET, 'labels/val', f'{class_name}_{i}.txt')
            with open(label_file, 'w') as f:
                f.write(f"{class_id} 0.5 0.5 1.0 1.0\n")
            val_count += 1

print(f"\n✅ Dataset conversion complete!")
print(f"  📊 Train: {train_count} images")
print(f"  📊 Val: {val_count} images")
print(f"  ⚠️  Skipped corrupt: {skipped}")
print(f"  💡 Limited to {MAX_IMAGES_PER_CLASS} images per class for faster training")

## Bước 5: Create YAML Config

In [ ]:
yaml_content = f"""path: {YOLO_DATASET}
train: images/train
val: images/val

nc: 7
names:
  0: Data caries
  1: Mouth Ulcer
  2: Tooth Discoloration
  3: hypodontia
  4: Gingivitis
  5: Calculus
  6: Caries_Gingivitus_ToothDiscoloration_Ulcer
"""

yaml_path = '/kaggle/working/data.yaml'
with open(yaml_path, 'w') as f:
    f.write(yaml_content)

print("✅ YAML config created")

# Verify
train_imgs = len(list(Path(f'{YOLO_DATASET}/images/train').glob('*.*')))
train_labels = len(list(Path(f'{YOLO_DATASET}/labels/train').glob('*.txt')))
val_imgs = len(list(Path(f'{YOLO_DATASET}/images/val').glob('*.*')))
val_labels = len(list(Path(f'{YOLO_DATASET}/labels/val').glob('*.txt')))

print(f"\n📊 Dataset verification:")
print(f"  Train: {train_imgs} images, {train_labels} labels {'✅' if train_imgs == train_labels else '❌'}")
print(f"  Val:   {val_imgs} images, {val_labels} labels {'✅' if val_imgs == val_labels else '❌'}")

if train_imgs == 0 or val_imgs == 0:
    print("\n⚠️  WARNING: No training or validation images!")

## Bước 6: Train YOLOv8

In [ ]:
print("🚀 Starting YOLOv8 training...\n")

# FIX: Disable PyTorch 2.6+ weights_only mode completely
import torch.serialization

# Only patch if not already patched (prevent recursion)
if not hasattr(torch.load, '_patched'):
    original_load = torch.load
    def patched_load(f, map_location=None, **kwargs):
        kwargs['weights_only'] = False
        return original_load(f, map_location=map_location, **kwargs)
    patched_load._patched = True
    torch.load = patched_load

# GPU info
gpus = tf.config.list_physical_devices('GPU')
print(f"🎮 GPUs: {len(gpus)}x T4")
print(f"📊 Dataset: {train_count + val_count} images")
print(f"⏱️  Epochs: 30")
print(f"🔥 Batch: 16\n")

# Clear cache
torch.cuda.empty_cache()

# Load model
print("📥 Loading YOLOv8n...")
model = YOLO('yolov8n.pt')

# Train - OPTIMIZED for fast epoch start
print("🔄 Training started...\n")
results = model.train(
    data=yaml_path,
    epochs=30,
    imgsz=640,
    batch=16,
    device=0,
    name='dental_detection',
    patience=8,
    save=True,
    save_period=-1,
    verbose=True,
    cache=False,      # DISABLED - No caching for immediate epoch start
    plots=True,
    amp=False,
    workers=0,        # CHANGED to 0 - Avoid multiprocessing overhead
    hsv_h=0.0,
    hsv_s=0.0,
    hsv_v=0.0,
    degrees=0,
    translate=0,
    scale=0.0,
    flipud=False,
    fliplr=False,
    mosaic=0.0
)

print("\n✅ Training completed!")

## Bước 7: Test Model

In [ ]:
print("📊 Testing model on validation images...\n")

best_model_path = '/kaggle/working/runs/detect/dental_detection/weights/best.pt'
model = YOLO(best_model_path)

# Get 10 validation images
val_images = list(Path(f'{YOLO_DATASET}/images/val').glob('*.jpg'))[:10]

fig, axes = plt.subplots(2, 5, figsize=(20, 8))
axes = axes.flatten()

for idx, img_path in enumerate(val_images):
    results = model(str(img_path), conf=0.5, verbose=False)
    annotated = results[0].plot()
    
    axes[idx].imshow(cv2.cvtColor(annotated, cv2.COLOR_BGR2RGB))
    axes[idx].set_title(f'{len(results[0].boxes)} detections')
    axes[idx].axis('off')

plt.tight_layout()
plt.savefig('/kaggle/working/detection_test.png', dpi=100, bbox_inches='tight')
plt.show()

print("✅ Test visualization saved!")

## Bước 8: Export Model

In [ ]:
print("💾 Exporting model...\n")

src_model = '/kaggle/working/runs/detect/dental_detection/weights/best.pt'
dst_model = '/kaggle/working/dental_detection_yolo.pt'

if os.path.exists(src_model):
    shutil.copy2(src_model, dst_model)
    size_mb = os.path.getsize(dst_model) / (1024 * 1024)
    print(f"✅ Model exported: dental_detection_yolo.pt")
    print(f"📦 Size: {size_mb:.2f} MB")
else:
    print("❌ Model not found!")

# Save classes
class_file = '/kaggle/working/yolo_classes.txt'
with open(class_file, 'w') as f:
    for name, id in class_mapping.items():
        f.write(f"{id}: {name}\n")
print(f"✅ Classes saved: yolo_classes.txt")

print("\n" + "="*60)
print("🎉 TRAINING COMPLETED!")
print("="*60)
print("\n📥 Files ready for download (Output tab):")
print("  1. dental_detection_yolo.pt - Main model")
print("  2. yolo_classes.txt - Class mapping")
print("  3. detection_test.png - Test results")
print("\n💾 Copy dental_detection_yolo.pt to models/ folder")